In [40]:
import gradio as gr
import numpy as np
import tensorflow as tf
from PIL import Image
import io
import requests
import tempfile
import os

# Function to download and load the model from GitHub
def load_model_from_github():
    # Raw file URL from GitHub (change 'blob' to 'raw' in the URL)
    model_url = "https://raw.githubusercontent.com/DatenFluss/machine-learning-university-course/main/res/models/font_recognition_cnn.keras"
    
    try:
        # Download the model file
        response = requests.get(model_url, allow_redirects=True)
        if response.status_code == 200:
            # Create a temporary file to save the model
            with tempfile.NamedTemporaryFile(delete=False, suffix='.keras') as tmp_file:
                tmp_file.write(response.content)
                tmp_file_path = tmp_file.name
            
            # Load the model from the temporary file
            model = tf.keras.models.load_model(tmp_file_path)
            
            # Clean up the temporary file
            os.unlink(tmp_file_path)
            
            return model
        else:
            raise Exception(f"Failed to download model: Status code {response.status_code}")
    except Exception as e:
        print(f"Error loading model from GitHub: {str(e)}")
        return None

# Load the model from GitHub
model = load_model_from_github()

# Font class names (make sure these match exactly with what the model was trained on)
font_classes = [
    "AGENCY", "ARIAL", "BAITI", "BANKGOTHIC", "BASKERVILLE", "BAUHAUS", "BELL",
    "BERLIN", "BERNARD", "BITSTREAMVERA", "BLACKADDER", "BODONI", "BOOK",
    "BOOKMAN", "BRADLEY", "BRITANNIC", "BROADWAY", "BRUSH", "BUXTON", "CAARD",
    "CALIBRI", "CALIFORNIAN", "CALISTO", "CAMBRIA", "CANDARA", "CASTELLAR",
    "CENTAUR", "CENTURY", "CHILLER", "CITYBLUEPRINT", "COMIC",
    "COMMERCIALSCRIPT", "COMPLEX", "CONSOLAS", "CONSTANTIA", "COOPER",
    "COPPERPLATE", "CORBEL", "COUNTRYBLUEPRINT", "COURIER", "CREDITCARD",
    "CURLZ", "DUTCH801", "E13B", "EBRIMA", "EDWARDIAN", "ELEPHANT", "ENGLISH",
    "ENGRAVERS", "ERAS", "EUROROMAN", "FELIX TITLING", "FOOTLIGHT", "FORTE",
    "FRANKLIN", "FREESTYLE", "FRENCH", "GABRIOLA", "GADUGI", "GARAMOND",
    "GEORGIA", "GIGI", "GILL", "GLOUCESTER", "GOTHICE", "GOUDY", "GUNPLAY",
    "HAETTENSCHWEILER", "HARLOW", "HARRINGTON", "HIGH TOWER", "HIMALAYA",
    "IMPACT", "IMPRINT", "INFORMAL", "ISOC", "ITALIC", "JAVANESE", "JOKERMAN",
    "JUICE", "KRISTEN", "KUNSTLER", "LEELAWADEE", "LUCIDA", "MAGNETO",
    "MAIANDRA", "MATURA", "MINGLIU", "MISTRAL", "MODERN", "MONEY",
    "MONOSPAC821", "MONOTXT", "MONOTYPE", "MV_BOLI", "MYANMAR", "NIAGARA",
    "NINA", "NIRMALA", "NUMERICS", "ONYX", "PALACE", "PALATINO", "PANROMAN",
    "PAPYRUS", "PERPETUA", "PHAGSPA", "PLAYBILL", "PMINGLIU-EXTB", "PRISTINA",
    "PROXY", "QUICKTYPE", "RAGE", "RAVIE", "REFERENCE", "RICHARD", "ROCKWELL",
    "ROMAN", "ROMANTIC", "SANSSERIF", "SCRIPT", "SCRIPTB", "SERIF",
    "SHOWCARD", "SIMPLEX", "SITKA", "SKETCHFLOW", "SNAP", "STENCIL", "STYLUS",
    "SUPERFRENCH", "SWIS721", "SYLFAEN", "TAHOMA", "TAI", "TECHNIC",
    "TEMPUS", "TIMES", "TREBUCHET", "TW", "TXT", "VERDANA", "VIN", "VINER",
    "VINETA", "VIVALDI", "VLADIMIR", "WIDE", "YI BAITI"
]


def preprocess_image(image):
    try:
        # Convert to grayscale if image is RGB
        if len(np.array(image).shape) == 3:
            image = Image.fromarray(np.array(image)).convert('L')
        
        # Resize to 20x20 pixels
        image_resized = image.resize((20, 20), Image.Resampling.LANCZOS)
        
        # Convert to numpy array and normalize
        img_array = np.array(image_resized).astype('float32')
        img_array = img_array / 255.0
        
        # Reshape for model input
        img_array = img_array.reshape(1, 20, 20, 1)
        
        return img_array
    except Exception as e:
        print(f"Error in preprocessing: {str(e)}")
        return None

def predict_font(input_image):
    try:
        if input_image is None:
            return None
        
        # Preprocess the image
        processed_image = preprocess_image(input_image)
        if processed_image is None:
            return None
            
        # Make prediction
        predictions = model.predict(processed_image, verbose=0)
        
        # Get top 3 predictions
        top_3_indices = np.argsort(predictions[0])[-3:][::-1]
        top_3_probabilities = predictions[0][top_3_indices]
        
        # Create dictionary of results
        results = {
            font_classes[idx]: float(prob) 
            for idx, prob in zip(top_3_indices, top_3_probabilities)
        }
        
        return results
    
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        return None

# Create Gradio interface
iface = gr.Interface(
    fn=predict_font,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="Font Recognition",
    description="Upload a character image to identify its font. The model will predict the top 3 most likely fonts.",
    examples=[],
    cache_examples=False
)

# Launch the interface
if __name__ == "__main__":
    iface.launch()

* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
